<a href="https://colab.research.google.com/github/bayileyegnabate/16-BigData/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.c

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-11 19:04:30--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-07-11 19:04:30 (6.49 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles

# Read in the Review dataset as a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [7]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8EWA1OFT84NX|          5|            0|          0|   N|                Y|
|R2JWY4YRQD4FOP|          5|            0|          0|   N|                N|
| RL5ESX231LZ0B|          5|            0|          0|   N|                Y|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|                Y|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|                Y|
|R13EPSFP5DODN5|          4|            0|          0|   N|                Y|
| R6RBP4HTE67SY|          5|            0|          0|   N|                Y|
|R15B3EU40RSU2W|          5|            0|          0|   N|                Y|
| RP4DD53A4ZJA2|          5|            0|          0|   N|                Y|
|R2C99DJEO4RZ4K|          5|            3|          4|   N|     

In [55]:
# retrieve all the rows where the total_votes >= 20
vine_df = vine_df.filter(vine_df["total_votes"] >= 20)
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|
|R3OIDSQJ84W7J1|          5|           21|         24|   N|                N|
|R3UUR313K5VVTL|          1|            1|         24|   N|                Y|
| RODE8K12S7148|          5|           41|         45|   N|     

In [56]:
from pyspark.sql.functions import format_number, count

# calculate the helpful_votes to total ratio
helpful_total_df = vine_df.withColumn("helpful_to_total_votes", format_number(vine_df["helpful_votes"] / vine_df["total_votes"], 2))
helpful_total_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_to_total_votes|
+--------------+-----------+-------------+-----------+----+-----------------+----------------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|                  0.91|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|                  1.00|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|                  0.99|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|                  0.96|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|                  0.87|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|                  0.83|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|              

In [57]:
# retrieve rows where the number of helpful_votes / total_votes >= 50%

helpful_total_df = helpful_total_df.filter(helpful_total_df["helpful_to_total_votes"] >= 0.50)
helpful_total_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_to_total_votes|
+--------------+-----------+-------------+-----------+----+-----------------+----------------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|                  0.91|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|                  1.00|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|                  0.99|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|                  0.96|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|                  0.87|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|                  0.83|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|              

In [58]:
# retrieves all rows where a review was written as part of the Vine program
vine_paid_df = helpful_total_df.filter(helpful_total_df["vine"] == "Y")
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_to_total_votes|
+--------------+-----------+-------------+-----------+----+-----------------+----------------------+
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|                  0.87|
| RSA6JQ346JZHZ|          5|           55|         64|   Y|                Y|                  0.86|
|R1FXF4HRMCLG4C|          5|           69|         84|   Y|                N|                  0.82|
| RCTBWC3II42MG|          4|          113|        117|   Y|                N|                  0.97|
| RTMQM2CQ1XIZ0|          5|           21|         25|   Y|                N|                  0.84|
|R2CBJLCKQ612KU|          4|           27|         35|   Y|                N|                  0.77|
|R322QDGO4AV2B5|          5|           55|         65|   Y|                N|              

In [59]:
# retrieves all rows where a review was written NOT as part of the Vine program
vine_unpaid_df = helpful_total_df.filter(helpful_total_df["vine"] == "N")
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_to_total_votes|
+--------------+-----------+-------------+-----------+----+-----------------+----------------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|                  0.91|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|                  1.00|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|                  0.99|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|                  0.96|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|                  0.83|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|                  0.95|
|R3OIDSQJ84W7J1|          5|           21|         24|   N|                N|              

In [64]:
# Note: the following results are based on the data filtered for rows
# at least 20 total reviews  

#total number of reviews
total_reviews = vine_df.count()

# total 5-star reviews
five_star_reviews = vine_df.filter(vine_df["star_rating"] == 5).count()

# paid 5-star reviews
paid_five_star_reviews = vine_paid_df.filter(vine_paid_df["star_rating"] == 5).count()

# percentage of paid 5-star reviews
percetage_paid_five_star_reviews = paid_five_star_reviews / five_star_reviews

# paid 5-star reviews
unpaid_five_star_reviews = vine_unpaid_df.filter(vine_unpaid_df["star_rating"] == 5).count()

# percentage of paid 5-star reviews
percetage_unpaid_five_star_reviews = unpaid_five_star_reviews / five_star_reviews

print(total_reviews, five_star_reviews, paid_five_star_reviews, percetage_paid_five_star_reviews, unpaid_five_star_reviews, percetage_unpaid_five_star_reviews)

27389 12371 202 0.016328510225527445 12033 0.9726780373454046
